In [1]:
import pandas as pd
import numpy as np
from utils.df_handle import *

In [3]:
df = pd.read_csv('practice_groupby.csv')

Just adding to the answers already given, the solution using the string "nunique"
seems much faster, tested here on ~21M rows dataframe, then grouped to ~2M

%time _=g.agg({"id": lambda x: x.nunique()})
CPU times: user 3min 3s, sys: 2.94 s, total: 3min 6s
Wall time: 3min 20s

%time _=g.agg({"id": pd.Series.nunique})
CPU times: user 3min 2s, sys: 2.44 s, total: 3min 4s
Wall time: 3min 18s

%time _=g.agg({"id": "nunique"})
CPU times: user 14 s, sys: 4.76 s, total: 18.8 s
Wall time: 24.4 s

In [6]:
# https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html

In [16]:
'min', 'max', 'size', 'count', 'nunique', 'sum', 'mean', 'median'

('min', 'max', 'count', 'nunique', 'sum', 'mean', 'median')

In [13]:
df.groupby(['Brand']).nunique()

,Sales
Brand,
A,5
B,5


In [17]:
df.groupby(['Brand']).agg(['min', 'max', 'count', 'nunique', 'sum', 'mean', 'median'])

Sales                                       
        min max count nunique sum      mean median
Brand                                             
A         1   5     6       5  20  3.333333    3.5
B         6  10     6       5  50  8.333333    8.5

In [21]:
%time PreGroupby_df = df.groupby(["Group1","Group2","Group3"]).agg({'Sales': 'sum'}).reset_index()
%time SumGroup_df = df.groupby(["Group1","Group2"]).agg({'Sales': 'sum'}).add_suffix('_Sum').reset_index()
%time Percents_df = pd.merge(PreGroupby_df, SumGroup_df)
%time Percents_df["Group3_Percentage"] = Percents_df["Sales"] / Percents_df["Sales_Sum"] * 100

Wall time: 7 ms
Wall time: 5.34 ms
Wall time: 3 ms
Wall time: 1e+03 µs


In [ ]:
# Initial grouping (basically a sorted version of df)
PreGroupby_df = df.groupby(["Group 1","Group 2","Final Group"]).agg({'Numbers I want as percents': 'sum'}).reset_index()
# Get the sum of values for the "final group", append "_Sum" to it's column name, and change it into a dataframe (.reset_index)
SumGroup_df = df.groupby(["Group 1","Group 2"]).agg({'Numbers I want as percents': 'sum'}).add_suffix('_Sum').reset_index()
# Merge the two dataframes
Percents_df = pd.merge(PreGroupby_df, SumGroup_df)
# Divide the two columns
Percents_df["Percent of Final Group"] = Percents_df["Numbers I want as percents"] / Percents_df["Numbers I want as percents_Sum"] * 100
# Drop the extra _Sum column
Percents_df.drop(["Numbers I want as percents_Sum"], inplace=True, axis=1)

#https://stackoverflow.com/questions/23377108/pandas-percentage-of-total-with-groupby/46105516#46105516

In [11]:
c =  df.groupby(["Group1","Group2","Group3"])["Sales"].sum()

Group1  Group2  Group3
A       HCM     HCM1       1
                HCM2       2
                HCM3       3
                Q1         1
                Q2         2
                Q3         3
        HN      HN1        8
                HN2       10
        HP      HP1       10
B       HCM     HCM1      12
                HCM2      14
                HCM3      16
        HN      HN1       18
                HN2       20
        HP      HP1       20
Name: Sales, dtype: int64

In [25]:
df = pd.read_csv('practice_groupby.csv')

In [26]:
%time c =  df.groupby(["Group1","Group2","Group3"])["Sales"].sum()
%time c / c.groupby(level=[0, 1]).transform("sum")
%time c.reset_index()

Wall time: 4.41 ms
Wall time: 2.03 ms
Wall time: 1.99 ms


,Group1,Group2,Group3,Sales
0,A,HCM,HCM1,1
1,A,HCM,HCM2,2
2,A,HCM,HCM3,3
3,A,HCM,Q1,1
4,A,HCM,Q2,2
5,A,HCM,Q3,3
6,A,HN,HN1,8
7,A,HN,HN2,10
8,A,HP,HP1,10
9,B,HCM,HCM1,12


In [1]:
url = (
    "https://raw.github.com/pandas-dev"
    "/pandas/master/pandas/tests/io/data/csv/tips.csv"
)

In [17]:
tips = pd.read_csv('practice_groupby.csv')
print(tips.shape)

(24, 4)


In [18]:
tips.isna().sum()

Group1    0
Group2    1
Group3    0
Sales     0
dtype: int64

In [19]:
tips.groupby(["Group1"]).agg({"Group2": np.size})

,Group2
Group1,
A,12
B,12


In [20]:
tips.groupby(["Group1"]).agg({"Group2": 'count'})
# Count returning the number of NOT NULL records within each.

,Group2
Group1,
A,12
B,11


In [15]:
tips.groupby(["smoker", "day"]).agg({"tip": ['count', np.mean]}).to_clipboard()

In [2]:
df1 = pd.read_csv('groupby_maxdate.csv', dayfirst=True, parse_dates=['DateOfAction', 'OrderDate'])

In [3]:
df1.dtypes

OrderDate       datetime64[ns]
DateOfAction    datetime64[ns]
Sales                    int64
dtype: object

In [4]:
pivot(df1, ['OrderDate'], {'DateOfAction':np.max})

,OrderDate,DateOfAction
0,2021-01-13,2021-01-17


In [34]:
df1 = pd.read_csv('sales_order.csv')

In [35]:
df2 = pd.read_csv('visit.csv')

In [36]:
df1.head()

,brand,city,DayofWeek,Created_Date
0,A,HCM,Mon,01/02/2021
1,A,HCM,Sun,02/02/2021
2,C,HCM,Mon,03/02/2021
3,C,HCM,Sun,04/02/2021
4,D,HN,Mon,05/02/2021


In [37]:
df2.head(20)

,brand,state,DayofWeek,StartDate,EndDate
0,A,HCM,Tue,02/02/2021,03/02/2021
1,A,HCM,Sun,01/02/2021,03/03/2021
2,A,HCM,Sun,04/02/2021,06/02/2021
3,C,HCM,Tue,03/02/2021,05/02/2021
4,C,HCM,Sun,04/02/2021,06/02/2021
5,E,HCM,Sun,07/02/2021,08/02/20212
6,E,HCM,Mon,09/02/2021,10/02/20212


In [98]:
df3 = pd.merge(df1, df2.add_prefix('df2_'), how='left', left_on=['brand', 'city'], right_on=['df2_brand', 'df2_state'])

In [95]:
# ctr1=df3.Created_Date<df3.df2_StartDate
# ctr2=df3.Created_Date>df3.df2_EndDate
# df3['RE']=np.where(ctr1|ctr2, True,False)
# df3['RE']

In [ ]:
dfb = pd.merge(dfb, dfa[['brand', 'city', 'Created_Date']], how='left',  on=['brand', 'city', 'Created_Date'], indicator=True)

In [106]:
def date_between_handler(df, ctr1, ctr2, series, list_check):
    """
    df = data frame
    ctr1, ctr2 = predefined, example: ctr1=df.Created_Date<df.df2_StartDate; ctr2=df1.Created_Date>df1.df2_EndDate
    series = df series should convert to np.NaN
    list_check = key joins + date columns (Created_Date) in bewteen (StartDate, EndDate): ['brand', 'city', 'Created_Date']
    """
    df['ERR']=np.where(ctr1|ctr2, True,False)
    print(f"row errors = {df['ERR'].sum()}")
    series = np.where(df['ERR'], np.NAN, series)

    # Chia df ra lam 2 phan a) Phan Loi b) Phan khong loi
    dfb = df[df['ERR']]
    dfa = df[~df['ERR']]
    # Check các dòng nằm trong phần lỗi có nằm trong phần không lỗi
    dfb = pd.merge(dfb, dfa[list_check], how='left',  on=list_check, indicator=True)
    # Chỉ giữ lại các dòng không nằm trong phần không lỗi
    ctr1 = dfb['_merge']=='left_only'
    dfb = dfb[ctr1]
    # Loại bỏ các dòng lỗi nhưng bị dup va cac dong khong can thiet
    dfb = dropdup(dfb,1,list_check).drop(labels=['ERR','_merge'], axis=1)
    df = dfa.append(dfb).drop(labels=['ERR'], axis=1)
    return df

In [107]:
ctr1=df3.Created_Date<df3.df2_StartDate
ctr2=df3.Created_Date>df3.df2_EndDate
series = df3['df2_brand']
list_check = ['brand', 'city', 'Created_Date']
df3 = date_between_handler(df3,ctr1,ctr2,series,list_check)

row errors = 6


In [85]:
df3

,brand,city,DayofWeek,Created_Date,df2_brand,df2_state,df2_DayofWeek,df2_StartDate,df2_EndDate
1,A,HCM,Mon,01/02/2021,A,HCM,Sun,01/02/2021,03/03/2021
3,A,HCM,Sun,02/02/2021,A,HCM,Tue,02/02/2021,03/02/2021
4,A,HCM,Sun,02/02/2021,A,HCM,Sun,01/02/2021,03/03/2021
6,C,HCM,Mon,03/02/2021,C,HCM,Tue,03/02/2021,05/02/2021
8,C,HCM,Sun,04/02/2021,C,HCM,Tue,03/02/2021,05/02/2021
9,C,HCM,Sun,04/02/2021,C,HCM,Sun,04/02/2021,06/02/2021
10,D,HN,Mon,05/02/2021,NaN,NaN,NaN,NaN,NaN
11,D,HN,Tue,06/02/2021,NaN,NaN,NaN,NaN,NaN
0,E,HCM,Sat,06/02/2021,NaN,HCM,Sun,07/02/2021,08/02/20212


In [82]:
df3[ctr4&(~ctr1|~ctr2|~ctr3)]

NameError: name 'ctr4' is not defined

In [105]:
ctr1=df3.df2_DayofWeek=='Sun'
ctr2=df3.Created_Date>=df3.df2_StartDate
ctr3=df3.Created_Date<=df3.df2_EndDate
ctr4=df3['_merge']=='both'
EXCLUDE=df3[ctr4&(~ctr1|~ctr2|~ctr3)][['df2_brand','df2_state','df2_DayofWeek','df2_StartDate','df2_EndDate']].copy()
EXCLUDE.drop_duplicates(inplace=True)
EXCLUDE.head()

,df2_brand,df2_state,df2_DayofWeek,df2_StartDate,df2_EndDate
0,A,HCM,Tue,02/02/2021,03/02/2021
2,A,HCM,Sun,04/02/2021,06/02/2021
6,C,HCM,Tue,03/02/2021,05/02/2021
7,C,HCM,Sun,04/02/2021,06/02/2021
12,E,HCM,Thu,04/02/2021,06/02/2021


In [ ]:
df4 = pd.merge(df2.add_prefix('df2_'), EXCLUDE, on=['df2_brand', 'df2_state', 'df2_DayofWeek', 'df2_StartDate','df2_EndDate'], how='outer', indicator=True)
df2=df4[~(df4['_merge']=='both')]

In [95]:
df3 = pd.merge(df1, df2, how='left', left_on=['brand', 'city'], right_on=['df2_brand', 'df2_state'])

In [96]:
df3

,brand,city,DayofWeek,Created_Date,df2_brand,df2_state,df2_DayofWeek,df2_StartDate,df2_EndDate,_merge
0,A,HCM,Mon,01/02/2021,A,HCM,Sun,01/02/2021,03/03/2021,left_only
1,A,HCM,Sun,02/02/2021,A,HCM,Sun,01/02/2021,03/03/2021,left_only
2,C,HCM,Mon,03/02/2021,NaN,NaN,NaN,NaN,NaN,NaN
3,C,HCM,Sun,04/02/2021,NaN,NaN,NaN,NaN,NaN,NaN
4,D,HN,Mon,05/02/2021,NaN,NaN,NaN,NaN,NaN,NaN
5,D,HN,Tue,06/02/2021,NaN,NaN,NaN,NaN,NaN,NaN
6,E,HCM,Sat,06/02/2021,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
ctr1=df3.df2_DayofWeek!='Sun'
ctr2=df3.Created_Date<df3.df2_StartDate
ctr3=df3.Created_Date>df3.df2_EndDate
ctr4=df3['_merge']=='both'
EXCLUDE = df3[ctr4 & (ctr1|ctr2|ctr3)][['df2_brand','df2_state','df2_DayofWeek','df2_StartDate','df2_EndDate']]


In [110]:
EXCLUDE.drop_duplicates(inplace=True)
EXCLUDE

,df2_brand,df2_state,df2_DayofWeek,df2_StartDate,df2_EndDate
0,A,HCM,Tue,02/02/2021,03/02/2021
2,A,HCM,Sun,04/02/2021,06/02/2021
6,C,HCM,Tue,03/02/2021,05/02/2021
7,C,HCM,Sun,04/02/2021,06/02/2021
12,E,HCM,Thu,04/02/2021,06/02/2021


In [51]:
df2.columns

Index(['brand', 'state', 'DayofWeek', 'StartDate', 'EndDate'], dtype='object')

In [57]:
onlist = EXCLUDE.columns

In [59]:
df4 = pd.merge(df2.add_prefix('df2_'), EXCLUDE, on=['df2_brand', 'df2_state', 'df2_DayofWeek', 'df2_StartDate','df2_EndDate'], how='outer', indicator=True)
df2=df4[~(df4['_merge']=='both')]

In [63]:
df2

,df2_brand,df2_state,df2_DayofWeek,df2_StartDate,df2_EndDate,_merge
1,A,HCM,Sun,01/02/2021,03/03/2021,left_only
5,E,HCM,Sun,04/02/2021,06/02/2021,left_only


In [29]:
df3

,brand,city,DayofWeek,Created_Date,df2_brand,df2_state,df2_DayofWeek,df2_StartDate,df2_EndDate,_merge
0,A,HCM,Mon,01/02/2021,A,HCM,Tue,02/02/2021,03/02/2021,both
1,A,HCM,Mon,01/02/2021,A,HCM,Sun,01/02/2021,03/03/2021,both
2,A,HCM,Mon,01/02/2021,A,HCM,Sun,04/02/2021,06/02/2021,both
3,A,HCM,Sun,02/02/2021,A,HCM,Tue,02/02/2021,03/02/2021,both
4,A,HCM,Sun,02/02/2021,A,HCM,Sun,01/02/2021,03/03/2021,both
5,A,HCM,Sun,02/02/2021,A,HCM,Sun,04/02/2021,06/02/2021,both
6,C,HCM,Mon,03/02/2021,C,HCM,Tue,03/02/2021,05/02/2021,both
7,C,HCM,Mon,03/02/2021,C,HCM,Sun,04/02/2021,06/02/2021,both
8,C,HCM,Sun,04/02/2021,C,HCM,Tue,03/02/2021,05/02/2021,both
9,C,HCM,Sun,04/02/2021,C,HCM,Sun,04/02/2021,06/02/2021,both
